Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (8933, 28, 28) (8933,)
Test set (8676, 28, 28) (8676,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (8933, 784) (8933, 10)
Test set (8676, 784) (8676, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Implementing L2 regularization in logistic regression.

In [5]:
batch_size = 128
learning_rate = 0.5
beta = 0.005

graph = tf.Graph()
with graph.as_default():
    # Input data: the training data will be fed at run-time with a training minibatch
    tf_train_dataset = tf.placeholder(tf.float32,
                                     shape=(batch_size, image_size * image_size))
    tf_train_label = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computations
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_label))
    
    # Add regularization term to the loss
    loss += beta * tf.nn.l2_loss(weights)
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    # Prediction for training, validation and test data
    train_predictions = tf.nn.softmax(logits)
    valid_predictions = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_predictions = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [6]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print ('Initialized')
    for step in range(num_steps):
        # Pick a randomized offset
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch
        batch_data = train_dataset[offset:(offset+batch_size), :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        # Prepare a dictionary to tell the session where to feed the minibatch
        # The key of the dictionary is the placeholder node of the graph to be fed
        # and the value is the numpy array to be fed
        feed_dict = {tf_train_dataset: batch_data, tf_train_label: batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_predictions], feed_dict=feed_dict)
        if (step % 500 == 0):
            print ('Minibatch loss as step %d: %f' %(step, l) )
            print ('Minibatch accuracy: %.1f' % accuracy(predictions, batch_labels))
            print ('Validation set accuracy: %.1f' % accuracy(valid_predictions.eval(), valid_labels))
    print ('Test set accuracy: %.1f' % accuracy(test_predictions.eval(), test_labels))
    session.close()

Initialized
Minibatch loss as step 0: 32.665215
Minibatch accuracy: 9.4
Validation set accuracy: 9.1
Minibatch loss as step 500: 1.550401
Minibatch accuracy: 82.0
Validation set accuracy: 78.5
Minibatch loss as step 1000: 0.832021
Minibatch accuracy: 78.9
Validation set accuracy: 80.7
Minibatch loss as step 1500: 0.525852
Minibatch accuracy: 83.6
Validation set accuracy: 80.6
Minibatch loss as step 2000: 0.605569
Minibatch accuracy: 89.1
Validation set accuracy: 80.6
Minibatch loss as step 2500: 0.742677
Minibatch accuracy: 80.5
Validation set accuracy: 80.5
Minibatch loss as step 3000: 0.741481
Minibatch accuracy: 82.8
Validation set accuracy: 80.6
Test set accuracy: 88.1


Implementing this for the neural network models

In [7]:
batch_size = 128
hidden_nodes = 1024
learning_rate = 0.5
beta = 0.001

graph = tf.Graph()

with graph.as_default():
    # Input data: the training data will be fed at run-time with a training minibatch
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_label = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    hidden_weights = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    hidden_biases = tf.Variable(tf.zeros([hidden_nodes]))
    
    weights = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training 
    def forward_prop(input):
        h1 = tf.nn.relu(tf.matmul(input, hidden_weights) + hidden_biases)
        return tf.matmul(h1, weights) + biases
    logits = forward_prop(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_label))
    
    # Regularized loss
    loss += beta * (tf.nn.l2_loss(hidden_weights) + tf.nn.l2_loss(weights))
    
    # Optimized
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    # Predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(forward_prop(tf_valid_dataset))
    test_prediction = tf.nn.softmax(forward_prop(tf_test_dataset))

In [8]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Pick a randomized offset
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size)]
        # Prepare a dictionary to tell the session where to feed the minibatch
        # The key of the dictionary is the placeholder node of the graph to be fed
        # and the value is the numpy array to be fed
        feed_dict = {tf_train_dataset: batch_data, tf_train_label:batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print ('Minibatch loss as step %d: %f' %(step, l) )
            print ('Minibatch accuracy: %.1f' % accuracy(predictions, batch_labels))
            print ('Validation set accuracy: %.1f' % accuracy(valid_prediction.eval(), valid_labels))
    print ('Test set accuracy: %.1f' % accuracy(test_prediction.eval(), test_labels))
    session.close()

Initialized
Minibatch loss as step 0: 707.909241
Minibatch accuracy: 6.2
Validation set accuracy: 26.2
Minibatch loss as step 500: 200.744507
Minibatch accuracy: 82.0
Validation set accuracy: 75.9
Minibatch loss as step 1000: 114.564789
Minibatch accuracy: 78.9
Validation set accuracy: 80.3
Minibatch loss as step 1500: 68.299767
Minibatch accuracy: 88.3
Validation set accuracy: 81.6
Minibatch loss as step 2000: 41.078724
Minibatch accuracy: 90.6
Validation set accuracy: 84.1
Minibatch loss as step 2500: 25.107609
Minibatch accuracy: 89.8
Validation set accuracy: 84.6
Minibatch loss as step 3000: 15.432660
Minibatch accuracy: 85.2
Validation set accuracy: 85.5
Test set accuracy: 92.5


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Pick a randomized offset
        offset = batch_size * np.random.choice(np.arange(5))
        # Generate a minibatch
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size)]
        # Prepare a dictionary to tell the session where to feed the minibatch
        # The key of the dictionary is the placeholder node of the graph to be fed
        # and the value is the numpy array to be fed
        feed_dict = {tf_train_dataset: batch_data, tf_train_label:batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print ('Minibatch loss as step %d: %f' %(step, l) )
            print ('Minibatch accuracy: %.1f' % accuracy(predictions, batch_labels))
            print ('Validation set accuracy: %.1f' % accuracy(valid_prediction.eval(), valid_labels))
    print ('Test set accuracy: %.1f' % accuracy(test_prediction.eval(), test_labels))
    session.close()

Initialized
Minibatch loss as step 0: 733.948303
Minibatch accuracy: 8.6
Validation set accuracy: 27.6
Minibatch loss as step 500: 190.137833
Minibatch accuracy: 100.0
Validation set accuracy: 74.7
Minibatch loss as step 1000: 115.310257
Minibatch accuracy: 100.0
Validation set accuracy: 74.7
Minibatch loss as step 1500: 69.930405
Minibatch accuracy: 100.0
Validation set accuracy: 74.8
Minibatch loss as step 2000: 42.409710
Minibatch accuracy: 100.0
Validation set accuracy: 74.7
Minibatch loss as step 2500: 25.720098
Minibatch accuracy: 100.0
Validation set accuracy: 74.8
Minibatch loss as step 3000: 15.599587
Minibatch accuracy: 100.0
Validation set accuracy: 75.0
Test set accuracy: 82.6


This is overfitting as this fits the training dataset very accurately but not so for validation and test dataset

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
